In [1]:
# !pip install nest_asyncio \
#     langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters \
#     python-pptx==1.0.2 nltk==3.9.1 pymupdf lxml \
#     sentence-transformers IProgress \
#     huggingface_hub ipywidgets \
#     qdrant-client

In [1]:

import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

In [3]:
hf_username = getpass.getpass("Enter Your Hugging Face Username: ")


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '67624d1b57e77fe6e0c87ae5', 'name': 'drewgenai', 'fullname': 'Drew DeMarco', 'email': 'drewgenai@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/L6eLaZmCK4jqW3ZTLYIAR.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'newotken', 'role': 'write', 'createdAt': '2025-02-12T04:11:04.130Z'}}}


In [6]:
!mkdir example_files
!mkdir output

mkdir: cannot create directory ‘example_files’: File exists
mkdir: cannot create directory ‘output’: File exists


In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

path = "example_files/"
text_loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)

1️⃣ Header-Based Chunking (Title-Based Splitter)
Uses document structure to split on headings, section titles, or patterns.
Works well for structured documents with named assessments, numbered lists, or headers.
Example: If it detects Chronic Pain Adjustment Index (CPAI-10), it groups everything under that title.
2️⃣ Semantic Chunking (Text-Meaning Splitter)
Uses embeddings or sentence similarity to decide where to break chunks.
Prevents splitting mid-context if sentences are closely related.
Example: Groups all related pain-assessment questions into one chunk.

In [8]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 200,
#     chunk_overlap  = 20,
#     length_function = len
# )


### potentially use for lenth tokens later

In [9]:
# #Load documents with metadata
# all_documents = text_loader.load()
# documents_with_metadata = []

In [10]:
# for doc in all_documents:
#     # Extract document name (assuming PyMuPDFLoader stores the file name in metadata)
#     source_name = doc.metadata.get("source", "unknown")
    
#     # Split into chunks while preserving metadata
#     chunks = text_splitter.split_documents([doc])
#     for chunk in chunks:
#         chunk.metadata["source"] = source_name  # Attach source info to each chunk
#     documents_with_metadata.extend(chunks)

###testingbelow


In [11]:
#!pip install langchain_experimental

In [12]:
from langchain_experimental.text_splitter import SemanticChunker

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings
model_id = "Snowflake/snowflake-arctic-embed-m"
embedding_model = HuggingFaceEmbeddings(model_name=model_id)
# model_id = "Snowflake/snowflake-arctic-embed-m-v2.0"
# embedding_model = HuggingFaceEmbeddings(model_name=model_id, model_kwargs={"trust_remote_code": True})


semantic_splitter = SemanticChunker(embedding_model)

all_documents = text_loader.load()
documents_with_metadata = []



/tmp/ipykernel_456462/1110142159.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_id)


In [13]:
#verify working
# test_doc = all_documents[0].page_content if all_documents else ""
# test_chunks = semantic_splitter.split_text(test_doc)

# print(f"\n✅ Total Chunks for First Document: {len(test_chunks)}")
# for i, chunk in enumerate(test_chunks[:3]):  # Show first 3 chunks
#     print(f"\n🔹 Chunk {i+1}: {chunk[:300]}")  # Print first 300 characters


In [14]:
from langchain.schema import Document

for doc in all_documents:
    source_name = doc.metadata.get("source", "unknown")  # Get document source

    # Use SemanticChunker to intelligently split text
    chunks = semantic_splitter.split_text(doc.page_content)

    # Convert chunks into LangChain Document format with metadata
    for chunk in chunks:
        doc_chunk = Document(page_content=chunk, metadata={"source": source_name})
        documents_with_metadata.append(doc_chunk)

###testingabove

In [15]:

#!pip install -qU huggingface_hub
#!pip install -qU ipywidgets


In [16]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings


# Load the SentenceTransformer model
model_id = "Snowflake/snowflake-arctic-embed-m"
embedding_model = HuggingFaceEmbeddings(model_name=model_id)

# Load documents into Qdrant
qdrant_vectorstore = Qdrant.from_documents(
    documents_with_metadata,
    embedding_model,
    location=":memory:",  # In-memory for testing
    collection_name="document_comparison",
)

# Create a retriever
qdrant_retriever = qdrant_vectorstore.as_retriever()

In [63]:
from langchain_core.prompts import ChatPromptTemplate
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question.

Return the response in **valid JSON format** with the following structure:

[
    {{
        "Derived Description": "A short name for the matched concept",
        "Protocol_1": "Protocol 1 - Matching Element",
        "Protocol_2": "Protocol 2 - Matching Element"
    }},
    ...
]

### Rules:
1. Only output **valid JSON** with no explanations, summaries, or markdown formatting.
2. Ensure each entry in the JSON list represents a single matched data element from the two protocols.
3. If no matching element is found in a protocol, leave it empty ("").
4. **Do NOT include headers, explanations, or additional formatting**—only return the raw JSON list.
5. It should include all the elements in the two protocols.
6. If it cannot match the element, create the row and include the protocol it did find and put "could not match" in the other protocol column.
7. protocol should be the between
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

from langchain_openai import ChatOpenAI

#openai_chat_model = ChatOpenAI(model="gpt-4o")
openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | openai_chat_model | StrOutputParser()
)

In [64]:
question_text = """You are a helpful assistant. Use the available context to answer the question.

Between these two files containing protocols, identify and match **entire assessment sections** based on conceptual similarity. Do NOT match individual questions.

### **Output Format:**
Return the response in **valid JSON format** structured as a list of dictionaries, where each dictionary contains:

[
    {
        "Derived Description": "A short name describing the matched sections",
        "Protocol_1": "Exact section heading from Protocol 1",
        "Protocol_2": "Exact section heading from Protocol 2"
    }
]

### **Matching Criteria:**
1. **Match entire assessment sections** based on their purpose and overall topic.
3. If a section in one protocol **has no match**, include it but leave the other protocol's field blank.
4. The **"Derived Description"** should be a **concise label** summarizing the section’s purpose, . It should describe the overall concept of the matched sections.

### **Rules:**
1. **Only output valid JSON**—no explanations, summaries, or markdown formatting.
2. **Ensure each entry represents a single section-to-section match.**
4. **Prioritize conceptual similarity over exact wording** when aligning sections.
5. If no match is found, leave the unmatched protocol entry blank.

### **Example Output:**
[
    {
        "Derived Description": "Pain Coping Strategies",
        "Protocol_1": "Pain Coping Strategy Scale (PCSS-9)",
        "Protocol_2": "Chronic Pain Adjustment Index (CPAI-10)"
    },
    {
        "Derived Description": "Work Stress and Fatigue",
        "Protocol_1": "Work-Related Stress Scale (WRSS-8)",
        "Protocol_2": "Occupational Fatigue Index (OFI-7)"
    },
]

Do not add any additional text, explanations, or formatting—**only return the raw JSON list**.
"""



# The questions within elements will be similar between the two documents and can be used to match the elements.

# 1. Derived description from the two documents describing the index/measure/scale.
# 2. A column for each standard.
# 3. In the column for each name/version, the data element used to capture that description that will be the shortened item between ()

# There should only be one row for each scale/index/etc.
# The description should not be one of the questions but a name that best describes the similar data elements."""

response_text = rag_chain.invoke({"question": question_text})
# response = rag_chain.invoke({"question": question_text})

In [67]:
import json
import pandas as pd

def parse_rag_output(response_text):
    """Extract structured JSON data from the RAG response."""
    try:
        structured_data = json.loads(response_text)

        # Ensure similarity score is always included
        for item in structured_data:
            item.setdefault("Similarity Score", "N/A")  # Default if missing

        return structured_data
    except json.JSONDecodeError:
        print("Error: Response is not valid JSON.")
        return None

def save_to_csv(data, directory="./output", filename="matching_data_elements.csv"):
    """Save structured data to CSV."""
    if not data:
        print("No data to save.")
        return

    file_path = os.path.join(directory, filename)
    df = pd.DataFrame(data, columns=["Derived Description", "Protocol_1", "Protocol_2"])  # Ensure correct columns
    df.to_csv(file_path, index=False)
    print(f"✅ CSV file saved: {filename}")

# Run the pipeline
structured_output = parse_rag_output(response_text)
save_to_csv(structured_output)


In [54]:
# rag_chain.invoke({"question" : "Based on the types of questions asked under each heading.  can you identify the headings in one document that most closely match the second document.  list them e.g   paincoping/doc1  painstrategy/doc2"})

'[\n    {\n        "Derived Description": "Memory Recall",\n        "Protocol_1_Name": "I struggle to remember names and faces. (Scale: 0-3)",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Memory Retention",\n        "Protocol_1_Name": "I retain new information effectively.",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Mnemonic Techniques",\n        "Protocol_1_Name": "I practice mnemonic techniques to improve recall.",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Task Management Difficulty",\n        "Protocol_1_Name": "could not match",\n        "Protocol_2_Name": "I find it difficult to keep track of multiple responsibilities. (Scale: 0-3)"\n    },\n    {\n        "Derived Description": "Mental Fatigue in Problem-Solving",\n        "Protocol_1_Name": "could not match",\n        "Protocol_2_Name": "I get mentally fatigued quickly when prob

In [31]:
# rag_chain.invoke({"question" : "Based on the types of questions asked under each heading.  can you identify the headings in one document that most closely match the second document.  list them e.g   paincoping/doc1  painstrategy/doc2. these are example headings not the ones in the actual documents.  just list the matches not the rational.  Can you list multiple matches?"})

'[\n    {\n        "Derived Description": "Memory Recall",\n        "Protocol_1_Name": "I struggle to remember names and faces.",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Retaining Information",\n        "Protocol_1_Name": "I retain new information effectively.",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Mnemonic Techniques",\n        "Protocol_1_Name": "could not match",\n        "Protocol_2_Name": "I practice mnemonic techniques to improve recall."\n    },\n    {\n        "Derived Description": "Pain Management Preparation",\n        "Protocol_1_Name": "I mentally prepare myself before engaging in painful activities.",\n        "Protocol_2_Name": "could not match"\n    },\n    {\n        "Derived Description": "Pain Minimization Techniques",\n        "Protocol_1_Name": "I use relaxation techniques to minimize pain perception.",\n        "Protocol_2_Name": "could not match"\n    },